In [1]:
import copy
import json
import os
import time
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.special import logit
from torchvision import models, transforms

In [2]:
import alt_models
from setup_residual_totirr import (addOne, applySGDmodel, cvSGDH,
                                   fitSGDR_Huber, getEVEInd, getNormalize,
                                   getResid, getXy, handleStd)
from util.SW_Dataset_bakeoff_totirr import SW_Dataset

In [3]:
# Check if MPS is available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# For future tensor operations, remember to create or move tensors to this device
# Example: x = torch.randn(3, 3, device=device)

Using device: mps


In [4]:
root = Path("/Users/andrewrobbertz/__SOC_CODE__/_data_/SDO/")
download_dir = root / "downloads"
experiments_dir = root / "experiments"

assert download_dir.exists(), f"Download directory {download_dir} does not exist."
assert (
    experiments_dir.exists()
), f"Experiments directory {experiments_dir} does not exist."

In [5]:
data_root = experiments_dir
eve_root = data_root / "EVE"

In [6]:
target_path = data_root / "results_anet_3_bn_15_20250718_145902"
target_path.mkdir(parents=True, exist_ok=True)

In [7]:
config_dir = Path.cwd().parent / "config"
assert config_dir.exists(), f"Config directory {config_dir} does not exist."

# Get the first JSON config file in the directory
config_file = list(config_dir.glob("*.json"))[0]

cfg = {}
with open(config_file, "r") as f:
    cfg = json.load(f)

In [8]:
model_file = target_path / f"{config_file.stem}_model.pt"
assert model_file.exists(), f"Model file {model_file} does not exist."

In [9]:
preds_file = target_path / f"{config_file.stem}_preds.npy"

In [10]:
phase = "test"
phase_abr = "Te"

In [11]:
sw_net = torch.load(model_file, weights_only=False)

# Move model to the device
sw_net = sw_net.to(device)

In [12]:
EVE_path = data_root / "irradiance_30mn_residual_14ptot.npy"
EVE_path_orig = data_root / "irradiance_30mn_14ptot.npy"
assert EVE_path.exists(), f"EVE data file {EVE_path} does not exist."
assert EVE_path_orig.exists(), f"EVE original data file {EVE_path_orig}"

In [13]:
model = np.load(data_root / "residual_initial_model.npz")
feats = np.load(data_root / "mean_std_feats.npz")

In [14]:
def addOne(X):
    return np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)


XTe = addOne((feats["X" + phase_abr] - model["mu"]) / model["sig"])
initialPredict = np.dot(XTe, model["model"].T)

In [15]:
crop = False
flip = False
batch_size = 64
resolution = 256
crop_res = 240
zscore = cfg["zscore"]

In [16]:
if zscore:  # we apply whatever scaling if zscore is on
    aia_mean = np.load(data_root / "aia_sqrt_mean.npy")
    aia_std = np.load(data_root / "aia_sqrt_std.npy")
    aia_transform = transforms.Compose(
        [transforms.Normalize(tuple(aia_mean), tuple(aia_std))]
    )
else:  # we don't sqrt and just divide by the means. just need to trick the transform
    aia_mean = np.load(data_root / "aia_mean.npy")
    aia_std = np.load(data_root / "aia_std.npy")
    aia_transform = transforms.Compose(
        [transforms.Normalize(tuple(aia_mean), tuple(aia_std))]
    )

In [17]:
### Dataset & Dataloader for test

test_real = SW_Dataset(
    EVE_path=EVE_path_orig,
    AIA_root=str(data_root / "AIA") + "/",
    index_file=str(data_root) + "/",
    resolution=resolution,
    EVE_scale=cfg["eve_transform"],
    EVE_sigmoid=cfg["eve_sigmoid"],
    split=phase,
    AIA_transform=aia_transform,
    crop=crop,
    flip=flip,
    crop_res=crop_res,
    zscore=zscore,
    self_mean_normalize=True,
)

sw_datasets = {
    x: SW_Dataset(
        EVE_path=EVE_path,
        AIA_root=str(data_root / "AIA") + "/",
        index_file=str(data_root) + "/",
        resolution=resolution,
        EVE_scale=cfg["eve_transform"],
        EVE_sigmoid=cfg["eve_sigmoid"],
        split=x,
        AIA_transform=aia_transform,
        flip=flip,
        crop=crop,
        crop_res=crop_res,
        zscore=zscore,
        self_mean_normalize=True,
    )
    for x in [phase]
}
sw_dataloaders = {
    x: torch.utils.data.DataLoader(
        sw_datasets[x], batch_size=batch_size, shuffle=False, num_workers=8
    )
    for x in [phase]
}
dataset_sizes = {x: len(sw_datasets[x]) for x in [phase]}

Loaded test Dataset with 655 examples
Loaded test Dataset with 655 examples


In [18]:
DS = sw_datasets[phase]

In [19]:
# TEST MODEL

outputs = []
for batchI, (inputs, _) in enumerate(sw_dataloaders[phase]):
    if batchI % 20 == 0:
        print("%06d/%06d" % (batchI, len(sw_dataloaders[phase])))

    # Set to GPU
    inputs = inputs.to(device)

    # Run Model on Test Inputs
    output = sw_net(inputs)
    output = output.cpu().detach().numpy()
    outputs.append(output)

prediction = np.concatenate(outputs, axis=0)

000000/000011


In [20]:
def eve_unscale(y, mean, std, nonlinearity, sigmoid, zscore):

    if zscore:
        y = y * std + mean
        if sigmoid:
            y = logit(y)
        if nonlinearity == "sqrt":
            y = np.power(y, 2)
        elif nonlinearity == "log":
            y = np.expm1(y)
    else:
        y *= mean

    return y


prediction = prediction / 100
prediction_us = eve_unscale(
    prediction,
    DS.EVE_means,
    DS.EVE_stds,
    cfg["eve_transform"],
    cfg["eve_sigmoid"],
    zscore,
)

In [21]:
absErrorPointwise = np.abs(prediction_us - DS.EVE) / DS.EVE
absErrorPointwise[DS.EVE <= 0] = np.nan

NP = initialPredict + prediction_us

PR = np.abs(initialPredict - test_real.EVE) / test_real.EVE
PR[test_real.EVE < 0] = np.nan
absErrorLin = np.nanmean(PR, axis=0)

NPR = np.abs(NP - test_real.EVE) / test_real.EVE
NPR[test_real.EVE < 0] = np.nan
absError = np.nanmean(NPR, axis=0)

In [22]:
print("Initial")
summaryStats = "Min: %.4f Mean: %.4f Median: %.4f Max: %.4f" % (
    np.min(absErrorLin),
    np.mean(absErrorLin),
    np.median(absErrorLin),
    np.max(absErrorLin),
)
print(summaryStats)
print(absErrorLin)

Initial
Min: 0.0041 Mean: 0.0736 Median: 0.0118 Max: 0.8951
[0.01389342 0.00727156 0.03054737 0.00775948 0.00413974 0.01126448
 0.01842462 0.01735587 0.00771773 0.01175596 0.00620152 0.01278114
 0.04777002 0.01131847 0.89507912]


In [23]:
def getResid(y, yp, mask, flare=None, flarePct=0.975):
    resid = np.abs(y - yp)
    resid = resid / np.abs(y) * 100
    resid[mask] = np.nan
    if flare is None:
        return np.nanmean(resid, axis=0)
    else:
        N = y.shape[0]
        FeXX = y[:, 2]
        order = np.argsort(FeXX)
        cutoff = int(y.shape[0] * flarePct)
        if flare:
            keep = order[cutoff:]
        else:
            keep = order[:cutoff]
        return np.nanmean(resid[keep, :], axis=0)


def printErrors(R, eve_root):
    names = np.load(os.path.join(eve_root, "name.npy"), allow_pickle=True)
    inds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14]
    names = names[inds]
    names = np.append(names, "tot_irr_megsa")
    for i in range(R.shape[0]):
        if i != 0:
            print("; ", end=" "),
        print("%s: %.2f%%" % (names[i].strip(), R[i]), end=" ")
    print("")


def print_analysis(y, yp, mask, eve_root):
    print("Overall")
    printErrors(getResid(y, yp, mask), eve_root)
    print("Flare")
    printErrors(getResid(y, yp, mask, flare=True), eve_root)
    print("Non-Flare")
    printErrors(getResid(y, yp, mask, flare=False), eve_root)

In [24]:
print_analysis(test_real.EVE, initialPredict, test_real.EVE < 0, eve_root)

Overall
Fe XVIII: 1.39% ;  Fe VIII: 0.73% ;  Fe XX: 3.05% ;  Fe IX: 0.78% ;  Fe X: 0.41% ;  Fe XI: 1.13% ;  Fe XII: 1.84% ;  Fe XIII: 1.74% ;  Fe XIV: 0.77% ;  He II: 1.18% ;  Fe XV: 0.62% ;  He II: 1.28% ;  Fe XVI: 4.78% ;  Mg IX: 1.13% ;  tot_irr_megsa: 89.51% 
Flare
Fe XVIII: 1.29% ;  Fe VIII: 0.81% ;  Fe XX: 4.43% ;  Fe IX: 0.98% ;  Fe X: 0.39% ;  Fe XI: 1.22% ;  Fe XII: 2.03% ;  Fe XIII: 2.11% ;  Fe XIV: 0.94% ;  He II: 0.83% ;  Fe XV: 0.38% ;  He II: 1.03% ;  Fe XVI: 3.16% ;  Mg IX: 1.52% ;  tot_irr_megsa: 89.19% 
Non-Flare
Fe XVIII: 1.39% ;  Fe VIII: 0.72% ;  Fe XX: 3.02% ;  Fe IX: 0.77% ;  Fe X: 0.41% ;  Fe XI: 1.12% ;  Fe XII: 1.84% ;  Fe XIII: 1.73% ;  Fe XIV: 0.77% ;  He II: 1.18% ;  Fe XV: 0.63% ;  He II: 1.28% ;  Fe XVI: 4.82% ;  Mg IX: 1.12% ;  tot_irr_megsa: 89.52% 


In [25]:
summaryStats = "Min: %.4f Mean: %.4f Median: %.4f Max: %.4f" % (
    np.min(absError),
    np.mean(absError),
    np.median(absError),
    np.max(absError),
)
print(summaryStats)
print(absError)
print_analysis(test_real.EVE, NP, test_real.EVE < 0, eve_root)

Min: 0.0033 Mean: 0.0740 Median: 0.0139 Max: 0.8944
[0.01155196 0.00700821 0.03006637 0.0054195  0.00326835 0.00843606
 0.01546556 0.01626483 0.00993826 0.01394866 0.01395916 0.01517291
 0.05467673 0.0103037  0.89435785]
Overall
Fe XVIII: 1.16% ;  Fe VIII: 0.70% ;  Fe XX: 3.01% ;  Fe IX: 0.54% ;  Fe X: 0.33% ;  Fe XI: 0.84% ;  Fe XII: 1.55% ;  Fe XIII: 1.63% ;  Fe XIV: 0.99% ;  He II: 1.39% ;  Fe XV: 1.40% ;  He II: 1.52% ;  Fe XVI: 5.47% ;  Mg IX: 1.03% ;  tot_irr_megsa: 89.44% 
Flare
Fe XVIII: 1.13% ;  Fe VIII: 0.63% ;  Fe XX: 3.39% ;  Fe IX: 0.67% ;  Fe X: 0.24% ;  Fe XI: 1.14% ;  Fe XII: 1.87% ;  Fe XIII: 2.04% ;  Fe XIV: 1.12% ;  He II: 1.21% ;  Fe XV: 0.92% ;  He II: 1.38% ;  Fe XVI: 4.00% ;  Mg IX: 1.31% ;  tot_irr_megsa: 89.13% 
Non-Flare
Fe XVIII: 1.16% ;  Fe VIII: 0.70% ;  Fe XX: 3.00% ;  Fe IX: 0.54% ;  Fe X: 0.33% ;  Fe XI: 0.84% ;  Fe XII: 1.54% ;  Fe XIII: 1.62% ;  Fe XIV: 0.99% ;  He II: 1.40% ;  Fe XV: 1.41% ;  He II: 1.52% ;  Fe XVI: 5.51% ;  Mg IX: 1.02% ;  tot_irr_me

In [26]:
resFile = open("%s/%s.txt" % (target_path, config_file.stem), "w")
resFile.write(summaryStats + "\n")
for i in range(15):
    resFile.write("%.4f " % absError[i])
resFile.close()

np.save(preds_file, NP)